In [14]:
#import nessesary library for data pre-processing
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img

#Use ImageDataGenerator for data augmentation to get more data for training and rescaling validation and testing data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=50, zoom_range=0.4, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train', target_size=(244,244), batch_size=10, class_mode='binary')
valid_generator = test_datagen.flow_from_directory('valid', target_size=(244,244), batch_size=10, class_mode='binary')

Found 161 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [15]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(244, 244, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 242, 242, 16)      448       
_________________________________________________________________
activation_14 (Activation)   (None, 242, 242, 16)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 121, 121, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 119, 119, 32)      4640      
_________________________________________________________________
activation_15 (Activation)   (None, 119, 119, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 57, 57, 64)        18496     
__________

In [16]:
from keras.callbacks import ModelCheckpoint

#use ModelCheckpoint to save weights with the best accuracy
checkpointer = ModelCheckpoint(filepath='weights.best.h5py', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#train the network on train_generator dataset
model.fit_generator(train_generator, steps_per_epoch=161, epochs=7, validation_data=valid_generator, validation_steps=54,callbacks=[checkpointer], verbose=1)


Epoch 1/7
161/161 [==============================] - 273s 2s/step - loss: 0.8566 - acc: 0.7379 - val_loss: 0.5414 - val_acc: 0.7593

Epoch 00001: val_acc improved from -inf to 0.75926, saving model to weights.best.h5py
Epoch 2/7
161/161 [==============================] - 275s 2s/step - loss: 0.3596 - acc: 0.8590 - val_loss: 0.6026 - val_acc: 0.8333

Epoch 00002: val_acc improved from 0.75926 to 0.83333, saving model to weights.best.h5py
Epoch 3/7
161/161 [==============================] - 266s 2s/step - loss: 0.2729 - acc: 0.8876 - val_loss: 1.0829 - val_acc: 0.7037

Epoch 00003: val_acc did not improve from 0.83333
Epoch 4/7
161/161 [==============================] - 264s 2s/step - loss: 0.2223 - acc: 0.9149 - val_loss: 0.6090 - val_acc: 0.8519

Epoch 00004: val_acc improved from 0.83333 to 0.85185, saving model to weights.best.h5py
Epoch 5/7
161/161 [==============================] - 262s 2s/step - loss: 0.2414 - acc: 0.9193 - val_loss: 0.7646 - val_acc: 0.8704

Epoch 00005: val_acc 

In [18]:
def accuracy(path):
    
    #load the weights with the best accuracy value
    model.load_weights('weights.best.h5py')
    
    test_generator = test_datagen.flow_from_directory(path, target_size=(244,244), batch_size=10, class_mode='binary')
    
    #Evaluate the model on the test set
    scores = model.evaluate_generator(test_generator,54)
    return  scores[1]

In [19]:
#here as the argument to the accuracy use the path to your test data
print ("Acuraccy: ", accuracy('test'))

Found 54 images belonging to 2 classes.
Acuraccy:  0.8518518469461198


In [4]:
from keras.applications import VGG16

model = VGG16(weights="imagenet", include_top=False)

train_data = model.predict_generator(train_generator,len(train_generator)//10)
train_label = train_generator.class_indices.keys()

valid_data = model.predict_generator(valid_generator,len(valid_generator)//10)
valid_label = valid_generator.class_indices.keys()

In [12]:
transfer_model = Sequential()
transfer_model.add(Flatten(input_shape=train_data.shape[1:]))
transfer_model.add(Dense(256))
transfer_model.add(Activation('relu'))
transfer_model.add(Dropout(0.4))
transfer_model.add(Dense(1))
transfer_model.add(Activation('sigmoid'))

transfer_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               6422784   
_________________________________________________________________
activation_12 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
_________________________________________________________________
activation_13 (Activation)   (None, 1)                 0         
Total params: 6,423,041
Trainable params: 6,423,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def transfer_accuracy(path):
    
    #load the weights with the best accuracy value
    transfer_model.load_weights('weights.best.h5py')
    
    test_data_gen = test_datagen.flow_from_directory(path, target_size=(244,244), batch_size=10, class_mode='binary')
    test_data = model.predict_generator(test_data_gen,len(test_data_gen)//10)
    test_label = test_generator.class_indices.keys()
    #Evaluate the model on the test set
    transfer_scores = model.evaluate(test_data, test_label, batch_size=10)
    return  transfer_scores[1]

In [ ]:
#use ModelCheckpoint to save weights with the best accuracy
checkpointer = ModelCheckpoint(filepath='transfer_weights.best.h5py', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

#train the network on train_generator dataset
transfer_model.fit_generator(train_generator, steps_per_epoch=161, epochs=7, validation_data=valid_generator, validation_steps=54,callbacks=[checkpointer], verbose=1)


In [ ]:
print ("Acuraccy: ", transfer_accuracy('test'))